In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

import torch
import matplotlib.pyplot as plt
from src import models, data
from tqdm.auto import tqdm
import json

In [ ]:
device = "cuda:0"
mt = models.load_model("gptj", device=device)
print(
    f"dtype: {mt.model.dtype}, device: {mt.model.device}, memory: {mt.model.get_memory_footprint()}"
)

In [ ]:
mt.__dict__.keys()

In [ ]:
prompt = "The Space Needle is located in the city of"
tokenized = mt.tokenizer(prompt, return_tensors="pt", padding=True).to(mt.model.device)

In [ ]:
import baukit

with baukit.TraceDict(
    mt.model,
    mt.layer_names,
) as traces:
    output = mt.model(**tokenized)

In [ ]:
output.logits[0][-1].shape

In [ ]:
def untuple(x):
    if isinstance(x, tuple):
        return x[0]
    return x


def interpret_logits(mt, logits, top_k=10):
    token_ids = logits.topk(dim=-1, k=top_k).indices.squeeze().tolist()
    logit_values = logits.topk(dim=-1, k=top_k).values.squeeze().tolist()
    return [
        (mt.tokenizer.decode(t), round(v, 3)) for t, v in zip(token_ids, logit_values)
    ]


def logit_lens(
    mt,
    h,
    interested_tokens=[],
    get_proba=False,
):
    logits = mt.lm_head(h)
    logits = torch.nn.functional.softmax(logits, dim=-1) if get_proba else logits
    candidates = interpret_logits(mt, logits)
    interested_logits = {
        t.item(): (logits[t].item(), mt.tokenizer.decode(t)) for t in interested_tokens
    }
    return candidates, interested_logits


# interpret_logits(mt, output.logits[0][-1])

In [ ]:
# interested_words = [" Seattle", " Paris", " Dhaka"]
# int_tokenized = mt.tokenizer(interested_words, return_tensors="pt", padding=True).to(
#     mt.model.device
# )
# int_tokenized.input_ids

# z = untuple(traces[mt.layer_names[-1]].output)[0][-1]
# print(z.shape)

# logit_lens(mt, z, [t[0] for t in int_tokenized.input_ids], get_proba=False)

In [ ]:
def filter_by_model_knowledge(mt, relation_prompt, relation_samples):
    model_knows = []
    for sample in relation_samples:
        tokenized = relation_prompt.format(sample.subject)
        output = mt.model(
            **mt.tokenizer(tokenized, return_tensors="pt", padding=True).to(
                mt.model.device
            )
        )

        object_id = output.logits[0][-1].argmax().item()
        object = mt.tokenizer.decode(object_id)

        tick = sample.object.strip().startswith(object.strip())
        # print(object, sample.object, tick)

        if tick:
            model_knows.append(sample)

    return model_knows

In [ ]:
dataset = data.load_dataset()
capital_cities = dataset[0]
capital_cities.__dict__.keys()

In [ ]:
import numpy as np

icl_indices = np.random.choice(range(len(capital_cities.samples)), 3, replace=False)
icl_samples = [capital_cities.samples[i] for i in icl_indices]
icl_prompt = [
    f"{capital_cities.prompt_templates[0].format(sample.subject)} {sample.object}"
    for sample in icl_samples
]
icl_prompt = "\n".join(icl_prompt) + "\n" + capital_cities.prompt_templates[0]

print(icl_prompt)

In [ ]:
model_knows = filter_by_model_knowledge(mt, icl_prompt, capital_cities.samples)
len(model_knows)

## Layer Richness based on logit lens

In [ ]:
def layer_richness(mt, relation_prompt, subject, verbose=False):
    tokenized = relation_prompt.format(subject)
    with baukit.TraceDict(mt.model, layers=mt.layer_names) as traces:
        output = mt.model(
            **mt.tokenizer(tokenized, return_tensors="pt", padding=True).to(
                mt.model.device
            )
        )

    object_id = output.logits[0][-1].argmax().item()
    object = mt.tokenizer.decode(object_id)
    # base_logit = output.logits[0][-1][object_id].item()
    base_score = torch.nn.functional.softmax(output.logits[0][-1], dim=-1)[
        object_id
    ].item()

    if verbose:
        print(f"object ==> {object} [{object_id}], base = {base_score}")

    layer_contributions = {}

    prev_score = 0
    for layer in mt.layer_names:
        h = untuple(traces[layer].output)[0][-1]
        candidates, interested_logits = logit_lens(
            mt, h, torch.tensor([object_id]), get_proba=True
        )
        layer_score = interested_logits[object_id][0]
        cur_layer_contribution = (layer_score - base_score) / base_score

        layer_contributions[layer] = cur_layer_contribution

        if verbose:
            print(f"layer: {layer}, diff: {cur_layer_contribution}")

        prev_score = layer_score

    return layer_contributions


relation_prompt = mt.tokenizer.eos_token + " {} is located in the city of"
subject = "The Space Needle"
layer_richness(mt, relation_prompt, subject, verbose=True)

In [ ]:
layer_richness_info = {layer: [] for layer in mt.layer_names}

for sample in tqdm(model_knows):
    cur_richness = layer_richness(mt, icl_prompt, sample.subject)
    for layer in mt.layer_names:
        layer_richness_info[layer].append(cur_richness[layer])

# with open("layer_sweep/layer_contribution_info.json", "w") as f:
with open("layer_sweep/layer_completeness_info.json", "w") as f:
    json.dump(layer_richness_info, f)

for layer in mt.layer_names:
    layer_richness_info[layer] = np.array(layer_richness_info[layer])

In [ ]:
mean_richness = [layer_richness_info[layer].mean() for layer in mt.layer_names]
low_richness = [layer_richness_info[layer].min() for layer in mt.layer_names]
high_richness = [layer_richness_info[layer].max() for layer in mt.layer_names]

plt.plot(mean_richness, color="blue")
plt.fill_between(range(len(mean_richness)), low_richness, high_richness, alpha=0.2)
plt.axhline(0, color="red", linestyle="--")

plt.xlabel("Layer")
plt.ylabel("completeness")
plt.xticks(range(0, len(mean_richness), 2))

plt.show()

## Layer Richness based on `Jh_norm` and `J_norm`

In [ ]:
import copy

capital_cities_known = copy.deepcopy(capital_cities.__dict__)
capital_cities_known["samples"] = model_knows

capital_cities_known = data.Relation(**capital_cities_known)

In [ ]:
from src.data import RelationSample
RelationSample(subject="Russia", object="Moscow")

In [ ]:
print(icl_prompt)

In [ ]:
from src.operators import JacobianEstimator, JacobianIclMeanEstimator

# mean_estimator = JacobianIclMeanEstimator(
#     mt=mt,
#     h_layer=12,
# )

# operator = mean_estimator(capital_cities_subset)
# operator("Russia", k = 10).predictions

In [ ]:
estimator = JacobianEstimator(
    mt=mt,
    h_layer=12,
)

operator = estimator.call_on_sample(
    sample = RelationSample(subject="Russia", object="Moscow"),
    prompt_template= icl_prompt
)

In [ ]:
operator.misc["Jh"].norm()

In [ ]:
relation_prompt = icl_prompt
subject = "Russia"
layer_richness(mt, relation_prompt, subject, verbose=True)

In [ ]:
layerwise_jh = {layer: [] for layer in mt.layer_names}

for sample in tqdm(set(model_knows) - set(icl_samples)):
    for h_layer in range(0, 24):
        layer_name = mt.layer_name_format.format(h_layer)
        estimator = JacobianEstimator(
            mt=mt,
            h_layer=h_layer,
        )
        operator = estimator.call_on_sample(
            # sample = RelationSample(subject="Russia", object="Moscow"),
            sample = sample,
            prompt_template= icl_prompt
        )

        # print(h_layer, " ===> ", f"J:{operator.weight.norm().item()},  Jh: {operator.misc['Jh'].norm().item()}")
        layerwise_jh[layer_name].append({
            "J": operator.weight.norm().item(),
            "Jh": operator.misc['Jh'].norm().item(),
            "bias": operator.bias.norm().item()
        })

In [ ]:
for layer in mt.layer_names:
    if layer in layerwise_jh and len(layerwise_jh[layer]) == 0:
        layerwise_jh.pop(layer)

with open("layer_sweep/layer_jh_info.json", "w") as f:
    json.dump(layerwise_jh, f)

In [ ]:
key = "Jh"

info = {
    layer: np.array([layerwise_jh[layer][i][key] for i in range(len(layerwise_jh[layer]))])
    for layer in layerwise_jh.keys()
}

mean = [info[layer].mean() for layer in info.keys()]
plt.plot(mean, color="blue", linewidth=4)
plt.xticks(range(0, len(mean), 2))
plt.ylabel(f"{key}_norm")

for i in range(len(set(model_knows) - set(icl_samples))):
    arr = []
    for layer in layerwise_jh.keys():
        arr.append(layerwise_jh[layer][i][key])
    plt.plot(arr, alpha=0.2)

plt.show()

## Layer sweep on mean ICL

In [ ]:
import copy

capital_cities_known = copy.deepcopy(capital_cities.__dict__)
capital_cities_known["samples"] = model_knows

capital_cities_known = data.Relation(**capital_cities_known)

In [ ]:
indices = np.random.choice(range(len(capital_cities_known.samples)), 2, replace=False)
samples = [capital_cities_known.samples[i] for i in indices]

capital_cities_subset = copy.deepcopy(capital_cities.__dict__)
capital_cities_subset["samples"] = samples
capital_cities_subset = data.Relation(**capital_cities_subset)

len(capital_cities_subset.samples)

In [ ]:
from src.operators import JacobianIclMeanEstimator

mean_estimator = JacobianIclMeanEstimator(
    mt=mt,
    h_layer=12,
)

operator = mean_estimator(capital_cities_subset)

In [ ]:
operator("Chile", k = 10).predictions

In [ ]:
predictions = []
target = []

for sample in tqdm(set(capital_cities_known.samples)):
    cur_predictions = operator(sample.subject, k = 5).predictions
    predictions.append([
        p.token for p in cur_predictions
    ])
    target.append(sample.object)

In [ ]:
from src.metrics import recall

recall(predictions, target)

In [ ]:
# np.savez("layer_sweep/operator_weight.npz", jacobian = operator.weight.detach().cpu().numpy(), allow_pickle=True)

In [ ]:
# j = np.load("layer_sweep/operator_weight.npz", allow_pickle=True)["jacobian"]

In [ ]:
# torch.dist(torch.tensor(j).to(device), operator.weight)

In [ ]:
def get_layer_wise_recall(capital_cities_subset, verbose = True, save_weights = True):

    layer_wise_recall = {}

    for h_layer in tqdm(range(0, 24)):
        layer_name = mt.layer_name_format.format(h_layer)
        mean_estimator = JacobianIclMeanEstimator(
            mt=mt,
            h_layer=h_layer,
        )
        operator = mean_estimator(capital_cities_subset)
        if(save_weights):
            np.savez(
                f"layer_sweep/weights_and_biases/{layer_name}.npz", 
                jacobian = operator.weight.detach().cpu().numpy(),
                bias = operator.bias.detach().cpu().numpy(), 
                allow_pickle=True
            )

        predictions = []
        target = []

        for sample in set(capital_cities_known.samples) - set(capital_cities_subset.samples):
            cur_predictions = operator(sample.subject, k = 5).predictions
            predictions.append([
                p.token for p in cur_predictions
            ])
            target.append(sample.object)

        layer_wise_recall[layer_name] = recall(predictions, target)
        
        if(verbose):
            print(layer_name, layer_wise_recall[layer_name])
    
    return layer_wise_recall

layer_wise_recall = get_layer_wise_recall(capital_cities_subset, verbose = True, save_weights = True)

In [ ]:
with open("layer_sweep/layer_wise_recall.json", "w") as f:
    json.dump(layer_wise_recall, f)

In [ ]:
layer_wise_recall_collection = {}
number_of_runs = 10

for run in tqdm(range(number_of_runs)):
    indices = np.random.choice(range(len(capital_cities_known.samples)), 2, replace=False)
    samples = [capital_cities_known.samples[i] for i in indices]

    capital_cities_subset = copy.deepcopy(capital_cities.__dict__)
    capital_cities_subset["samples"] = samples
    capital_cities_subset = data.Relation(**capital_cities_subset)

    layer_wise_recall = get_layer_wise_recall(capital_cities_subset, verbose=False, save_weights=False)

    for layer in layer_wise_recall.keys():
        if(layer not in layer_wise_recall_collection):
            layer_wise_recall_collection[layer] = []
        layer_wise_recall_collection[layer].append(layer_wise_recall[layer])

In [ ]:
with open("layer_sweep/layer_wise_recall_collection.json", "w") as f:
    json.dump(layer_wise_recall_collection, f)

In [ ]:
top_1 = np.array([
    np.array(layer_wise_recall_collection[layer])[:, 0]
    for layer in layer_wise_recall_collection.keys()
])
top_1.mean(axis = 1)

In [ ]:
# top_1 = [layer_wise_recall[layer][0] for layer in layer_wise_recall.keys()]
# top_2 = [layer_wise_recall[layer][1] for layer in layer_wise_recall.keys()]
# top_3 = [layer_wise_recall[layer][2] for layer in layer_wise_recall.keys()]

top_1 = np.array([
    np.array(layer_wise_recall_collection[layer])[:, 0]
    for layer in layer_wise_recall_collection.keys()
])

top_2 = np.array([
    np.array(layer_wise_recall_collection[layer])[:, 1]
    for layer in layer_wise_recall_collection.keys()
])

top_3 = np.array([
    np.array(layer_wise_recall_collection[layer])[:, 2]
    for layer in layer_wise_recall_collection.keys()
])




plt.plot(top_1.mean(axis=1), color="green", linewidth=3, label="recall@1")
plt.plot(top_2.mean(axis=1), color="blue", linewidth=2, label="recall@2")
plt.plot(top_3.mean(axis=1), color="red", linewidth=1, label="recall@3")

plt.fill_between(
    range(len(layer_wise_recall_collection.keys())),
    top_1.min(axis=1), top_1.max(axis=1),
    color="green", alpha=0.1
)

plt.xticks(range(0, len(top_1), 2))
plt.xlabel("layer")
plt.ylabel("recall")

plt.legend()
plt.show()

In [ ]:
from src.utils.misc import visualize_matrix

for h_layer in range(0, 24):
    layer_name = mt.layer_name_format.format(h_layer)
    j = np.load(f"layer_sweep/weights_and_biases/{layer_name}.npz", allow_pickle=True)["jacobian"]
    j = torch.tensor(j).to(device)
    print(layer_name, j.shape)
    visualize_matrix(j, title = layer_name, save_path=f"layer_sweep/Jacobian_plots/{layer_name}.png")